# 로컬 Ollama 모델 성능 테스트
* 모델명: ollama 3.1b:8b
* 3가지 질문: 1.텍스트 생성능력 2.요약 능력 3. 정형데이터 파악 능력 

In [1]:
import requests

def run_ollama(model: str, prompt: str) -> str:
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": model,
        "prompt": prompt,
        "stream": False
    }

    response = requests.post(url, json=payload)
    response.raise_for_status()
    return response.json()["response"]

# 생성능력

In [ ]:
model = "llama3.1:8b"
prompt = "협업의 중요성 주제로 짧은 글 써줘."
run_ollama(model, prompt) # 답변 시간 27초

'협업, 서로가 성장하는 공간입니다.\n\n일상생활과 함께 일하기도 어렵지만, 공동으로 문제를 해결해나가는 것은 협력을 통해 가능합니다. 사람들과의 협력은 결국 팀워크에 의미를 부여하며, 더 큰 것을 이루는 것입니다.'

In [ ]:
model = "llama3.1:8b"
prompt = ""
run_ollama(model, prompt) # 답변 시간 27초

# 요약 능력

In [ ]:
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

model = "llama3.1:8b"
llm = Ollama(model=model,
             temperature=0)

template = """
다음은 회의록 내용입니다. 핵심 내용을 간결하게 요약해주세요.

회의록 전문: 
{transcript}

요약:
"""

prompt = PromptTemplate(
    template= template,
    input_variables=["transcript"]
)

meeting_context = """[2025.07.29] 마케팅 전략 기획 회의록

참석자: 김대리, 이과장, 박부장, 최팀장

1. 2분기 매출 분석
- 김대리: 온라인 채널의 매출이 전년 대비 18% 증가했으며, 특히 20대 여성 타겟 제품의 반응이 좋았음.
- 이과장: 오프라인 매장은 매출이 6% 감소. 주중 방문객 수가 줄어든 것이 주요 원인으로 보임.

2. 하반기 캠페인 방향
- 박부장: SNS 중심의 브랜드 캠페인 기획 필요. 8월 중순부터 티저 콘텐츠 배포 시작 예정.
- 최팀장: 인플루언서 연계 프로모션도 고려 중. 타겟층은 20~30대 여성으로 집중.

3. 예산 관련 논의
- 김대리: 온라인 광고 집행 예산을 기존보다 15% 확대 요청.
- 이과장: 오프라인 홍보비는 유지하되, 운영 시간 단축 등으로 절감 가능성 있음.

4. 다음 회의 일정
- 다음 회의는 8월 5일 오후 2시로 결정됨.
"""


chain = LLMChain(llm=llm, prompt=prompt)
result = chain.invoke({"transcript": meeting_context})
print(result['text'])

2025년 7월 29일 마케팅 전략 기획 회의에서 논의된 주요 내용은 다음과 같습니다.

* 온라인 매출이 18% 증가했으며, 특히 20대 여성 타겟 제품의 반응이 좋았습니다.
* 오프라인 매장은 매출이 6% 감소했습니다. 주중 방문객 수가 줄어든 것이 주요 원인입니다.
* 하반기 캠페인은 SNS 중심의 브랜드 캠페인을 기획하고, 인플루언서 연계 프로모션도 고려합니다.
* 온라인 광고 집행 예산을 15% 확대 요청했습니다.


# 데이터처리 능력

In [9]:
from sklearn.datasets import load_iris, load_wine
import pandas as pd

data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [10]:
from langchain.llms import Ollama
from langchain_experimental.agents import create_pandas_dataframe_agent

model = "llama3.1:8b"
llm = Ollama(model=model,
             temperature=0)

agent = create_pandas_dataframe_agent(llm, 
                                      df, 
                                      verbose=True,
                                      allow_dangerous_code=True,
                                      handle_parsing_errors=True
                                      )

prompt = """
너는 최고의 데이터 분석가야. 
1. 어떤 데이터인지
2. 어떤 통계적 의미를 가지고 있는지
설명해줘.
"""
result = agent.run(prompt)
result 

c:\Users\21ckw\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\langchain_experimental\agents\agent_toolkits\pandas\base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(
C:\Users\21ckw\AppData\Local\Temp\ipykernel_22480\3014015688.py:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(prompt)




> Entering new AgentExecutor chain...
Thought: The data appears to be a dataset of flower measurements, specifically the sepal and petal lengths and widths.
Action: python_repl_ast
Action Input: print(df.info())<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB
None
It seems like you want me to explain the data and its statistical significance.

Action: python_repl_ast
Action Input: print(df.describe())       sepal length (cm)  sepal width (cm)  petal length (cm)  \
count         150.000000        150.000000         150.000000   
mean            5.843333          3.057333           3.758000   
std             

'이 데이터는 꽃의 측정치로 구성된 데이터셋입니다. Sepal과 Petal의 길이와 너비를 포함합니다.\n통계적으로, 이 데이터는 다음과 같은 의미를 가지고 있습니다:\n- Sepal length (cm)의 평균은 5.84 cm이고 표준편차는 0.83 cm입니다.\n- Sepal width (cm)의 평균은 3.06 cm이고 표준편차는 0.44 cm입니다.\n- Petal length (cm)의 평균은 3.76 cm이고 표준편차는 1.77 cm입니다.\n- Petal width (cm)의 평균은 1.20 cm이고 표준편차는 0.76 cm입니다.\n이 데이터는 꽃의 형태와 크기와 관련된 통계적 특징을 제공합니다.'